# Chinese Word Segmentation
Using tokenizer, padding and binary classification

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import keras_tuner as kt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras. preprocessing.sequence import pad_sequences

In [2]:
train = pd.read_csv('C:/Users/mandy/OneDrive/2022 IRS/ML/Datasets/Chinese Word Segmentation/as_training.utf8', names=['data'])
print(train.head())

                    data
0                   時間　：
1  三月　十日　（　星期四　）　上午　十時　。
2                   地點　：
3      學術　活動　中心　一樓　簡報室　。
4                   主講　：


## Create data and labels

In [3]:
y = train['data'].copy()
X = train['data'].str.replace('\u3000', '')
print(X.head())
print(type(X))
print(y.head())

0               時間：
1    三月十日（星期四）上午十時。
2               地點：
3      學術活動中心一樓簡報室。
4               主講：
Name: data, dtype: object
<class 'pandas.core.series.Series'>
0                     時間　：
1    三月　十日　（　星期四　）　上午　十時　。
2                     地點　：
3        學術　活動　中心　一樓　簡報室　。
4                     主講　：
Name: data, dtype: object


In [4]:
def create_labels(data):
    label = []
    index = 0

    while index < len(data) - 1:
        if data[index + 1] == '\u3000':
            label.append(1)
            index += 2
        else:
            label.append(0)
            index += 1
            
    if index == len(data) - 1:
        label.append(1)
    return label

y = y.apply(create_labels)

In [5]:
print(y)
print(type(y))

0                                          [0, 1, 1]
1         [0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1]
2                                          [0, 1, 1]
3               [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1]
4                                          [0, 1, 1]
                             ...                    
708948             [0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1]
708949                      [0, 1, 1, 1, 0, 0, 1, 1]
708950                                     [0, 1, 1]
708951    [0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1]
708952          [0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1]
Name: data, Length: 708953, dtype: object
<class 'pandas.core.series.Series'>


## Tokenization and Padding

In [6]:
tokenizer = Tokenizer(oov_token='<OOV>', split='\u3000', char_level=True)
tokenizer.fit_on_texts(train['data'])
char_index = tokenizer.word_index
total_chars = len(char_index) + 1

In [7]:
print(list(char_index.keys())[:100])
print(len(char_index))

['<OOV>', '\u3000', '，', '的', '。', '一', '是', '不', '有', '在', '人', '我', '了', '、', '這', '大', '以', '為', '個', '中', '他', '會', '來', '國', '上', '時', '要', '們', '就', '到', '生', '學', '「', '可', '」', '年', '也', '之', '說', '對', '出', '而', '能', '地', '成', '得', '家', '多', '所', '自', '十', '於', '後', '都', '子', '好', '：', '過', '下', '與', '用', '方', '作', '如', '小', '和', '你', '心', '因', '發', '行', '天', '經', '那', '然', '分', '動', '現', '前', '？', '日', '去', '事', '公', '三', '法', '∥', '麼', '及', '長', '很', '業', '！', '同', '當', '開', '民', '其', '但', '二']
6086


In [8]:
max_length = X.str.len().max() # 188
# trunc_type = 'post'
padding = 'pre'

X_sequences = tokenizer.texts_to_sequences(X)
padded_X = pad_sequences(X_sequences, maxlen=max_length, padding=padding)
padded_y = pad_sequences(y, maxlen=max_length, padding=padding)
print(padded_X[1])
print(padded_X.shape)

[ 85 181  51  81 115 572 221 164 114  25 721  51  26   5   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0]
(708953, 188)


In [23]:
embedding_dim = 256

xIn = Input((188))
x = Embedding(total_chars, embedding_dim, input_length=188, mask_zero=True)(xIn) # mask_zero will ignore timestamps with 0 (aka ignoring the padding)
x = Bidirectional(LSTM(64, return_sequences=True))(x)
x = Bidirectional(LSTM(64, return_sequences=True))(x)
# x = Flatten()(x) don't need to flatten, just put output layer as 2 neurons
# x = Dense(64, activation='swish')(x)
# x = Dense(64, activation='swish')(x)
x = Dense(64, activation='swish')(x)
x = Dense(64, activation='swish')(x)
xOut = Dense(2, activation='linear')(x) # treat as multiclass cause signmoid doesn't do well

model = Model(inputs=xIn, outputs=xOut)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 188)]             0         
                                                                 
 embedding_11 (Embedding)    (None, 188, 256)          1558272   
                                                                 
 bidirectional_16 (Bidirecti  (None, 188, 128)         164352    
 onal)                                                           
                                                                 
 bidirectional_17 (Bidirecti  (None, 188, 128)         98816     
 onal)                                                           
                                                                 
 flatten (Flatten)           (None, 24064)             0         
                                                                 
 dense_17 (Dense)            (None, 64)                1540

In [24]:
epochs = 30

callbacks = [
    tf.keras.callbacks.ModelCheckpoint('./8_best_model', monitor='acc', save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='acc', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='acc', factor=0.1, patience=3, verbose=1)
]

padded_X = tf.convert_to_tensor(padded_X)
padded_y = tf.convert_to_tensor(padded_y)
print(type(padded_y))
history = model.fit(padded_X, padded_y, batch_size=32, epochs=epochs, callbacks=callbacks, use_multiprocessing=True)

<class 'tensorflow.python.framework.ops.EagerTensor'>
Epoch 1/30
22154/22155 [============================>.] - ETA: 0s - loss: 0.0110 - acc: 0.0696

INFO:tensorflow:Assets written to: .\8_best_model\assets


INFO:tensorflow:Assets written to: .\8_best_model\assets


22155/22155 [==============================] - 1127s 50ms/step - loss: 0.0110 - acc: 0.0696 - lr: 0.0010
Epoch 2/30
22154/22155 [============================>.] - ETA: 0s - loss: 0.0054 - acc: 0.0902

INFO:tensorflow:Assets written to: .\8_best_model\assets


...

22155/22155 [==============================] - 1143s 52ms/step - loss: 0.0043 - acc: 0.0905 - lr: 0.0010
Epoch 4/30
22155/22155 [==============================] - ETA: 0s - loss: 0.0037 - acc: 0.0935

INFO:tensorflow:Assets written to: .\8_best_model\assets


INFO:tensorflow:Assets written to: .\8_best_model\assets


22155/22155 [==============================] - 1147s 52ms/step - loss: 0.0037 - acc: 0.0935 - lr: 0.0010
Epoch 5/30
 8014/22155 [=========>....................] - ETA: 11:51 - loss: 0.0032 - acc: 0.0909

KeyboardInterrupt: 

## Save Model

In [ ]:
import datetime as dt
dtime = dt.time()
now = dt.datetime.now()
now.strftime("%Y-%m-%d %H-%M-%S")

model.save(f'8_Chinese_Word_Segmentation/8_saved_models/{now}.h5')